## 누락된 데이터 다루기

### 테이블 형태의 데이터에서 누락된 값 식별

In [2]:
import pandas as pd
from io import StringIO

In [7]:
csv_data = \
"""
A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0, 6.0,,8.0
10.0,11.0,12.0,
"""

In [8]:
df = pd.read_csv(StringIO(csv_data))

In [9]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [12]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [14]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [15]:
df.dropna(how='all') # 모두 NaN일때 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [16]:
df.dropna(thresh=4) # 실수가 4보다 작을 때

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [17]:
df.dropna(subset=['C']) # 특정열에 NaN이 있는 경우에 삭제 (C열)

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 누락된 값 대체하기

In [21]:
import numpy as np
from sklearn.impute import SimpleImputer

In [23]:
simr = SimpleImputer(missing_values=np.nan, strategy='mean')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data


array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [24]:
from sklearn.preprocessing import FunctionTransformer

In [26]:
ftr_simr = FunctionTransformer(lambda X: simr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])